In [2]:
from model_prediction import combine_roots_and_themes
import pandas as pd

# Read the CSV file into a DataFrame
input = pd.read_csv('sentences_author.csv')
input = input.sample(n=20, random_state=42)
print(input)

In [ ]:
results = combine_roots_and_themes(input)
results

In [7]:
results.to_csv('attitude_roots_themes.csv', index=False)

In [7]:
def create_clusters(row):
    # Extract root and themes
    root = row["attitude_root"]
    themes = row["attitude_themes"]
    
    # Combine root with each theme
    clusters = [f"{root}({theme})" for theme in themes]
    return clusters

df = results
df.loc[:, "clusters"] = df.apply(create_clusters, axis=1)
df = df.explode("clusters", ignore_index=True)

In [8]:
df

In [14]:
# Group by 'author' and 'clusters', and aggregate the sentences into a list
aggregated_df = df.groupby(['clusters', 'author'])['sentence'].apply(list).reset_index()
aggregated_df

In [16]:
final_df = aggregated_df.groupby('clusters').agg(
            comments=('sentence', lambda x: [[author, sentences] for author, sentences in zip(aggregated_df['author'], x)])
        ).reset_index()

In [17]:
final_df